In [ ]:
serverPath = "data/Servers.txt"
patchesPath = "data/Patches.txt"
appliedPatchesPath = "data/AppliedPatches.txt"

outputPath1 = "outPart1_v2/"
outputPath2 = "outPart2_v2/"

In [ ]:
# Define the rdds associated with the used input files


# Input format: SID, OS, Model
serverRDD = sc.textFile(serverPath)

# Input format: PID, ReleaseDate, OS
patchesRDD = sc.textFile(patchesPath)

# Input format: PID, SID, ApplicationDate
appliedPatchesRDD = sc.textFile(appliedPatchesPath)

In [ ]:
#########################################
# PART 1
#########################################

In [ ]:
# First, select the patches associated with Ubuntu2.
# Then, map the patches RDD into a pair rdd:
# key = PID
# value = ReleaseDate

def mapPidRelDate(l):
    fields = l.split(",")
    pid = fields[0]
    releaseDate = fields[1]
    return (pid, releaseDate)
    

pidOsUbuntuRDD = patchesRDD.filter(lambda l: l.split(",")[2]=="Ubuntu2")\
                .map(mapPidRelDate)

In [ ]:
# from appliedPatches RDD obtain a pair RDD with
# key = PID
# value = AppliedDate

def mapPidApplDate(l):
    fields = l.split(",")
    pid = fields[0]
    applicationDate = fields[2]
    return (pid, applicationDate)
    

pidAppliedDateRDD = appliedPatchesRDD.map(mapPidApplDate)

In [ ]:
# Join the two RDDs so that (left = appliedPatches, right = patches)
# key = PID
# value = (application date, release date)
# then filter only those lines for which application date == release date
patchesAppliedAtRelease = pidAppliedDateRDD\
                            .join(pidOsUbuntuRDD)\
                            .filter(lambda t: t[1][0]==t[1][1])

In [ ]:
# Each element in patchesAppliedAtRelease represents a patch applied on a server with Ubuntu2
# at the release date
# Now map all the elements into a pairRDD with
# key = PID
# value = 1
# and use a reduceByKey to count for each patch the number of servers on which
# the patches were applied at release date. 
# Filter and keep only those patches which were applied to 100 servers or more
res1 = patchesAppliedAtRelease.map(lambda t: (t[0], 1)) \
                    .reduceByKey(lambda i1, i2: i1 + i2)\
                    .filter(lambda s: s[1] >= 100)

In [ ]:
# Store the selected PIDs in the first output folder 
res1.keys()\
    .saveAsTextFile(outputPath1)

In [ ]:
#########################################
# PART 2
#########################################

In [ ]:
# Starting from applied patches rdd, filter only those patches applied in 2021
# and map into a pair RDD with
# key = SID
# value = month
# and perform a distinct operation to keep for each server the distinct months

def filter2021(l):
    fields = l.split(",")
    date = fields[2]
    return date.startswith("2021")

def sidMonth(l):
    fields = l.split(",")
    sid = fields[1]
    date = fields[2]
    month = int(date.split("/")[1])
    return (sid, month)


serverMonthAppliedPatch = appliedPatchesRDD\
            .filter(filter2021)\
            .map(sidMonth)\
            .distinct()

In [ ]:
# Compute the number of distinct months with at least one applied patch for each server

serverMonths21NumApplPatches = serverMonthAppliedPatch.mapValues(lambda v: 1)\
                                                .reduceByKey(lambda v1, v2: v1+v2)

In [ ]:
# Calculate the number of distinct months in 2021 without at least one applied patch 
# for each server by applying the formula 12 - number of months with applied patches
serverMonths21NumNoApplPatches = serverMonths21NumApplPatches.mapValues(lambda v: 12-v)

In [ ]:
# serverMonths21NoApplPatches does not contain the servers without applied patches 
# for all the 12 months of 2021

# Use serversRDD to gather information from all servers

# Prepare a pairRDD with
# key = SID
# value = 12
def SID12(l):
    fields = l.split(",")
    sid = fields[0]
    return (sid, 12)



allServers = serverRDD.map(SID12)

In [ ]:
# Apply left outer join to join all servers with the information about the servers with 
# at least one month with applied patches
SID12Months21NumNoApplPatches = allServers\
                    .leftOuterJoin(serverMonths21NumNoApplPatches)

In [ ]:
# If the second part of the value is None it means that for that server 
# no patches have been applied in 2021 (no applied patches for all 12 months).
# Otherwise, the non-None value already is the number of months without applied patches.
# Map to the correct value depending on the second part of each input value
res2 = SID12Months21NumNoApplPatches\
        .mapValues(lambda v: 12 if v[1] is None else v[1])

In [ ]:
# Store the result in second output folder 
res2.saveAsTextFile(outputPath2)